In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
cd /content/drive/MyDrive/딥러닝 자연어처리

/content/drive/MyDrive/딥러닝 자연어처리


In [30]:
# sent_list=[['배에', '가스가', '차서', '방귀,', '트림', '등이', '생기는', '것이', '거짓증세는' ,'아니라는', '설명이다', '.'],
#            ['경찰청' ,'사이버수사국은','청소년의', '신종' ,'코로나바이러스' ,'감염증(코로나19)', '백신' ,'접종을' ,'앞두고', '과학적·실증적', '근거' ,'없이' ,'국민' ,'불안감을', '가중하는', '악의적·조직적' ,'허위조작정보' ,'유포', '및', '개인정보', '유출에', '대해' ,'엄정' ,'수사하겠다고' ,'30일', '밝혔다', '.'],
#            ['최근', '우리나라뿐', '아니라', '앞서', '전', '세계', '많은', '여성이', '백신', '접종', '시작', '이후', '온라인에서', '생리', '불순이나', '평소보다' ,'심한', '생리통이' ,'혹시' ,'알려지지', '않은' ,'코로나', '백신', '부작용인지', '묻기', '시작했다', '.']]
# tag_list=[['O', 'SYMP-B', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
#           [	'O' ,'O' ,'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',],
#           [	'O', 'O', 'O', 'O', 'O' ,'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'SYMP-B', 'SYMP-I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]

In [43]:
sent_list = [['아스트라제네카와', '얀센', '백신의', '희귀', '혈전증과', '화이자', '백신의', '심근염이', '새로', '발견된', '가능한', '부작용의', '예다', '.']]
tag_list = [['VACC-B', 'VACC-B', 'O', 'DIS-B', 'DIS-I', 'VACC-B', 'O', 'DIS-B', 'O', 'O', 'O', 'O', 'O', 'O']]

In [63]:
from itertools import combinations

def merge_tags(tags):
        prior = False
        prior_tag = False
        chunk_dict, chunk_id = dict(), 0
        for i, tag in enumerate(tags):
            bio = tag[-1]
            if bio == "B":
                if prior:
                    chunk_dict[chunk_id]['end'] = i - 1
                chunk_id += 1
                chunk_dict[chunk_id] = {"entity_type": tag[:-2], "begin": i, "end": i}
                prior = True
                prior_tag = chunk_dict[chunk_id]['entity_type']  # 'SYMP'
                #print(prior_tag)
            elif bio == "I":
                if (prior_tag == False) or (tag[:-2] != prior_tag) : # 앞전의 태그가 O였거나 다른 태그였던 경우
                    #print(prior_tag, bio)
                    chunk_id +=1
                    chunk_dict[chunk_id] = {"entity_type": tag[:-2], "begin": i, "end": i}
                    prior = True
                    prior_tag =chunk_dict[chunk_id]['entity_type'] 
                else: # 앞전의 태그와 동일한 태그가 이어서 나온 경우
                    chunk_dict[chunk_id]['end'] = i 
            elif bio == "O":
                if prior:
                    chunk_dict[chunk_id]['end'] = i - 1
                prior = False
                prior_tag = False
        return chunk_dict
    
def to_relation_format(sent_list, tag_list):
        result = []
        for sent, tags in zip(sent_list, tag_list):
            
            #print(sent,tags)
            chunk_dict = merge_tags(tags)
            #print(chunk_dict)
            if len(chunk_dict) < 2:  # 엔티티 갯수가 2개 이상일 때만 포함
                continue
            for subj, obj in combinations(chunk_dict, 2):
                #print(chunk_dict[subj], chunk_dict[obj])
                re_sent = sent.copy()
                subj, obj = chunk_dict[subj], chunk_dict[obj]
                subj_begin, subj_end, obj_begin, obj_end = subj['begin'], subj['end'], obj['begin'], obj['end']
                re_sent[subj_begin] = "<e1>" + re_sent[subj_begin]
                re_sent[subj_end] = re_sent[subj_end] + "</e1>"
                re_sent[obj_begin] = "<e2>" + re_sent[obj_begin]
                re_sent[obj_end] = re_sent[obj_end] + "</e2>"
                result.append((" ".join(re_sent), subj["entity_type"], obj["entity_type"]))
        return result


In [64]:
to_relation_format(sent_list, tag_list)

[('<e1>아스트라제네카와</e1> <e2>얀센</e2> 백신의 희귀 혈전증과 화이자 백신의 심근염이 새로 발견된 가능한 부작용의 예다 .',
  'VACC',
  'VACC'),
 ('<e1>아스트라제네카와</e1> 얀센 백신의 <e2>희귀 혈전증과</e2> 화이자 백신의 심근염이 새로 발견된 가능한 부작용의 예다 .',
  'VACC',
  'DIS'),
 ('<e1>아스트라제네카와</e1> 얀센 백신의 희귀 혈전증과 <e2>화이자</e2> 백신의 심근염이 새로 발견된 가능한 부작용의 예다 .',
  'VACC',
  'VACC'),
 ('<e1>아스트라제네카와</e1> 얀센 백신의 희귀 혈전증과 화이자 백신의 <e2>심근염이</e2> 새로 발견된 가능한 부작용의 예다 .',
  'VACC',
  'DIS'),
 ('아스트라제네카와 <e1>얀센</e1> 백신의 <e2>희귀 혈전증과</e2> 화이자 백신의 심근염이 새로 발견된 가능한 부작용의 예다 .',
  'VACC',
  'DIS'),
 ('아스트라제네카와 <e1>얀센</e1> 백신의 희귀 혈전증과 <e2>화이자</e2> 백신의 심근염이 새로 발견된 가능한 부작용의 예다 .',
  'VACC',
  'VACC'),
 ('아스트라제네카와 <e1>얀센</e1> 백신의 희귀 혈전증과 화이자 백신의 <e2>심근염이</e2> 새로 발견된 가능한 부작용의 예다 .',
  'VACC',
  'DIS'),
 ('아스트라제네카와 얀센 백신의 <e1>희귀 혈전증과</e1> <e2>화이자</e2> 백신의 심근염이 새로 발견된 가능한 부작용의 예다 .',
  'DIS',
  'VACC'),
 ('아스트라제네카와 얀센 백신의 <e1>희귀 혈전증과</e1> 화이자 백신의 <e2>심근염이</e2> 새로 발견된 가능한 부작용의 예다 .',
  'DIS',
  'DIS'),
 ('아스트라제네카와 얀센 백신의 희귀 혈전증과 <e1>화이자</e1> 백신의 <e2>심근염이</e2> 새로 발견된 가능한 부작용의 예다 .',
  'VACC'